# Text Modelling : TF-IDF, Cosine Similarity, BM-25

### TF-IDF
Dataset chosen is from a collection of about 7,000 Yelp reviews from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge).
Each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field.
I treat each review as a document. Given a query, you need to calculate its TF-IDF score in each review.

`TF = number of times word occurs in a document`

`IDF = log(total number of documents / number of documents containing the word)`

IDF has several formulations I have chosen the above for simplicity


In [1]:
import json
import math
data=[]
with open("review.json",'r') as json_file:
    for line in json_file:
        data.append(json.loads(line))
N=len(data) #Num of documents in the corpus 
#Example: First document
print(data[0])

{'review': 'every time i go horrible stale taco shells hard uncooked beans need i go on stay clear of this taco bell ', 'id': 'oN9FgM2DYrMjUKdrJS2lYQ'}


### Preprocessing
Preprocessing will help us mine tf and df for the entire vocabulary of the document corpus.
This will be used in later functions to implement TF-IDF, cosine and BM-25 methods.

In [2]:
#Preprocessing
#Precalculate df of all terms in all docs, later use this to calculate docmag for cosine scoring
df_dict=dict()
for doc in data:
    terms_unique=set(doc['review'].split())
    for term in terms_unique:
        df_dict[term]=df_dict.get(term,0)+1

#Calculate idf for all terms
idf_dict=dict()
for term in df_dict:
    idf_dict[term]=(math.log(N/float(df_dict[term])))
avgdl=0 #Average doc length initialise to 0
total_len=0 #Add doc lengths here
doc_info=dict()

#Preprocessing
#Go through all docs to get every doc's tf
for doc in data:
    docid=doc['id']
    terms=doc['review'].split()
    total_len+=len(terms)
    doc_dict=dict()
    docmag=0
    for term in terms:
        if term not in doc_dict:
            doc_dict[term]=terms.count(term)
    for term,tf in doc_dict.items():
        docmag+=(idf_dict[term]*tf)**2.
    docmag=math.sqrt(docmag)
    doc_info[docid]=(doc_dict,docmag)
#Get average doc length across the corpus for BM25
avgdl=total_len/N

#Following functionality splits testquery and stores doc scores for all docs for the query terms
def score_documents(testquery,scoring):
    query=testquery.split()
    idfarray=[0.] * len(query) #Initialiase idf array with zeros len = no of query terms
    doc_metric=dict() #Will contain only docs that have search queries and tf values
    #Next part helps calculate docmetric with key=docid and value depending on scoring type
    for doc in data:
        docid=doc['id'] #Get Doc id
        tfarray=[0] * len(query) 
        tfarray= [doc_info[docid][0].get(x,0) for x in query]
        if tfarray!=[0] * len(query): #Only store metric for doc with at least one query term
            if scoring=="tfidf_score":
                doc_metric[docid]=(tfarray) #Store tf array and 0 for compatibility
            elif scoring=="cosine_score":
                doc_metric[docid]=(tfarray,doc_info[docid][1]) #store tuple of tfarray of only search terms and docmag
            elif scoring=="bm25_score":
                doc_metric[docid]=(tfarray,sum(doc_info[docid][0].values())) #store tuple of tf array and doc length
    #Calculate IDF of only query terms
    idfarray=[idf_dict.get(q,0) for q in query]
    return doc_metric,idfarray

#Function to calculate tf*idf score
def tfidf_score():
    score=dict() #Key= Document ID Value=Sum of tf*idf score of each 
    #Calculate composite Sum of tf*idf score for each term
    for x in list(doc_metric): #iterate over all docs with non zero tf
        scr=0
        for i in range(len(doc_metric[x])):
            scr+=doc_metric[x][i]*idfarray[i]
        score[x]=scr
    return score 


### Print Top 10 documents that have highest score for the query


In [9]:
#Function to print results
def printresults():
    topten = sorted(composite_score.items(),key = lambda x:x[1], reverse=True)[:10]
    print("Query=","\"",testquery,"\"")
    print("Rank Document ID \t\t Score \n")
    i=1
    for x in topten:
        print(i,"{0} \t {1}".format(*x,))
        i+=1

### Example query and scoring method with top ten documents printed with scores


In [10]:
testquery="best bbq"
scoring="tfidf_score"
doc_metric,idfarray=score_documents(testquery,scoring)
composite_score=tfidf_score()
printresults()

Query= " best bbq "
Rank Document ID 		 Score 

1 YbQvHNrjZJ38mnh5rLuq7w 	 26.319774733191345
2 P31kXP4oan6ZQm69TN6tIA 	 21.933145610992785
3 x5esEK6J9XkA_vbvVbG8Gg 	 19.506448347290707
4 mWs26TrBM7ogwCM9UfVJFg 	 17.54651648879423
5 NCfX4AxDvQ3QRyXKtmhVwQ 	 17.54651648879423
6 e5INq6DAZn2zMHicKQl07Q 	 15.119819225092153
7 4WTG1-9mw8YHEyaTu8dQww 	 15.119819225092153
8 x3n_l3GhBx78y6jWX4fStg 	 13.719523009475362
9 Wp8jYXL1DQrgrnZIFmufFg 	 13.159887366595672
10 jrEx93eYKIjCW2nrkwjZpQ 	 13.159887366595672


### Ranking with TF-IDF + Cosine

Instead of using the sum of TF-IDF scores, now we still use the TF-IDF scores to weigh each term, but also cosine between the query vector and the document vector to assign a similarity score.

In [5]:
def cosine_score():
    score=dict()
    for x in list(doc_metric): #iterate over all docs with non zero tf for at least one search term
        scr=0
        #Syntax= doc_metric["docid"]=([tfarray],docmag)  doc_metric[x][1]=docmag
        y=len(doc_metric[x][0]) #no of terms
        for i in range(y):
            scr+=(doc_metric[x][0][i]*idfarray[i])*(1./math.sqrt(float(y)))*(1/doc_metric[x][1])
        score[x]=scr
    #Calculate composite Sum of tf*idf score for each term
    return score

In [6]:
testquery="best bbq"
scoring="cosine_score"
doc_metric,idfarray=score_documents(testquery,scoring)
composite_score=cosine_score()
printresults()

Query best bbq
Rank Document ID 		 Score 

1 6wRJtHhvQsS6vOse466_3w 	 0.5345284475881048
2 x5esEK6J9XkA_vbvVbG8Gg 	 0.4360447415638787
3 eAXFFP3GxUfGjQlAZDB_CQ 	 0.42255218023429036
4 7LaBODDEaUNRpLPDG_bKtQ 	 0.40004536886838066
5 P31kXP4oan6ZQm69TN6tIA 	 0.35771827655726385
6 ZAn6zB6VOCsJ1OsGRv-NVA 	 0.35459450721620783
7 8p-KEtrrTmLv-o1mKpUy1A 	 0.33997398367851134
8 RHWT1KVeIw2FT7i5BP_TVw 	 0.31605256633839557
9 _fNfowXaxXcYChKukMrYeg 	 0.308230511122904
10 AEiNkWY-4ggToDeMTd8l1w 	 0.2990933593101938


### Ranking with BM25

BM-25 Introduction [https://en.wikipedia.org/wiki/Okapi_BM25](https://en.wikipedia.org/wiki/Okapi_BM25) 
I have chosen k_1 = 1.2 and b = 0.75

In [7]:
def bm25_score():
    score=dict()
    k=1.2
    b=0.75
    for x in list(doc_metric): #iterate over all docs with non zero tf
        scr=0
        for i in range(len(doc_metric[x][0])):
            scr+=idfarray[i]*((doc_metric[x][0][i]*(1+k))/(doc_metric[x][0][i]+k*(1-b+b*(doc_metric[x][1])/avgdl)))
        score[x]=scr
    #Calculate composite Sum of tf*idf score for each term
    return score 

In [8]:
testquery="best bbq"
scoring="bm25_score"
doc_metric,idfarray=score_documents(testquery,scoring)
composite_score=bm25_score()
printresults()

Query best bbq
Rank Document ID 		 Score 

1 x5esEK6J9XkA_vbvVbG8Gg 	 9.721870551819865
2 xpm6TgDiHaQdEDlErFsqvQ 	 9.415118348525638
3 4WTG1-9mw8YHEyaTu8dQww 	 8.959421822569597
4 e5INq6DAZn2zMHicKQl07Q 	 8.592495681676898
5 GASAd_gPBY_eWIL9XJwuNA 	 7.976652147350818
6 P31kXP4oan6ZQm69TN6tIA 	 7.880072215237718
7 8p-KEtrrTmLv-o1mKpUy1A 	 7.619530382843498
8 HzNxErSCQ2FYfPCbyfHrSQ 	 7.436157918661526
9 -RApX_RMzJLnpommDpQfKQ 	 7.377703314704793
10 1tJ_iJX_KZ3zM_9_GRaGTg 	 7.193190780305126


### Discussion on the three methods:

TF*IDF method:
It is the simplest method to implement. However the top match for "best bbq" was just a review with "bbq" ie the rarer term, mentioned 6 times which is to be expected given that rarer terms have large IDF. There was no penalty for a longer document, top scoring doc is a 358 word review, it scores high only because of the rarer term mentioned more number of times, however this approch will skew results towards longer documents which will obviously mention a term more often. An approach with some doc normalisation will give better results.

Cosine Method: 
This method takes care of document normalisation by using the doc_mag (the length of the doc to normalise the tf*idf score).
Takes more computation because tf idf score is to be calculated for each term in each doc for normalisation.
Cosine method's highest rated review for "best bbq" is just a short document with one term ie "best" even though "best" has lower idf than "bbq". Short doc length is over weighted/preferred in cosine approach. This approach severely penalises documents that are slightly longer and chooses one of the shortest docs as the best.

BM-25: 
Seems to be the best method in terms of computation cycles and extracting most important document.
Does not involve either the computation intensive ways of cosine doc normalisation (normalisation is with simple doc length) nor is it heavily skewed in favor of rarer terms.
Best rated document for "best bbq" was a relatively short doc length of 110 words, with bbq freq=4 and best freq=1. This was the most pertinent and relevant doc for the given query.